In [1]:
from graphframes.lib import AggregateMessages as AM
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark import SparkContext
from graphframes import *
from operator import itemgetter
import matplotlib.pyplot as plt
import networkx as nx
import findspark
import pandas as pd

findspark.init()

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()
sc = spark.sparkContext

22/01/15 14:28:23 WARN Utils: Your hostname, Rose resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface enp3s0)
22/01/15 14:28:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/15 14:28:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/15 14:28:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/15 14:28:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.sql.functions import sum as sqlsum
from pyspark.sql.functions import collect_set
from graphframes.lib import AggregateMessages as AM

# Vertex DataFrame
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)
], ["id", "name", "age"])
# Edge DataFrame
e = spark.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)


In [4]:
# For each user, sum the ages of the adjacent users.
msgToSrc = AM.dst["age"]
msgToDst = AM.src["age"]
agg = g.aggregateMessages(
    collect_set(AM.msg).alias("summedAges"),
    sendToSrc=msgToSrc,
    sendToDst=msgToDst)
agg.show()

+---+------------+
| id|  summedAges|
+---+------------+
|  f|    [30, 32]|
|  e|[34, 36, 29]|
|  d|    [34, 32]|
|  c|        [36]|
|  b|    [30, 34]|
|  a|[32, 36, 29]|
+---+------------+



In [5]:
result = g.labelPropagation(maxIter=5)
result.select("id", "label").show()

+---+-------------+
| id|        label|
+---+-------------+
|  g| 146028888064|
|  b|1047972020224|
|  e|1382979469312|
|  a|1382979469312|
|  f|1460288880640|
|  d|1460288880640|
|  c|1382979469312|
+---+-------------+

